In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint

: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

: 

In [ ]:
device

: 

In [ ]:
with open("shakespeare.txt", "r") as file:
    content = file.read()

: 

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(content)))
stoi = {s:i for i,s in enumerate(chars)}
# stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
pprint(itos)

: 

In [ ]:
block_size = 4 # context length: how many characters do we take to predict the next one?
X, Y = [], []

  
#print(w)
context = [0] * block_size
for ch in content:
  ix = stoi[ch]
  X.append(context)
  Y.append(ix)
  print(''.join(itos[i] for i in context), '--->', itos[ix])
  context = context[1:] + [ix] # crop and append
  
# Move data to GPU

X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

: 

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

: 

In [ ]:
emb_dim = 2
emb = torch.nn.Embedding(len(stoi), emb_dim)
emb.weight
emb.weight.shape

: 

In [ ]:
# Function to visualize the embedding in 2d space

def plot_emb(emb, itos, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    for i in range(len(itos)):
        x, y = emb.weight[i].detach().cpu().numpy()
        ax.scatter(x, y, color='k')
        ax.text(x + 0.05, y + 0.05, itos[i])
    return ax

plot_emb(emb, itos)

: 

In [ ]:
class NextChar(nn.Module):
  def __init__(self, block_size, vocab_size, emb_dim, hidden_size):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, emb_dim)
    self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
    self.lin2 = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    x = self.emb(x)
    x = x.view(x.shape[0], -1)
    x = torch.sin(self.lin1(x))
    x = self.lin2(x)
    return x

: 

In [ ]:
# Generate Text from untrained model


model = NextChar(block_size, len(stoi), emb_dim, 10).to(device)
# model = torch.compile(model)

g = torch.Generator()
g.manual_seed(4000002)
def generate_text(model, itos, stoi, block_size, max_len=10):
    context = [0] * block_size
    txt = ''
    for i in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        ch = itos[ix]
        txt += ch
        context = context[1:] + [ix]
    return txt

print(generate_text(model, itos, stoi, block_size,30))

: 

In [ ]:
for param_name, param in model.named_parameters():
    print(param_name, param.shape)

: 

In [ ]:
#code for training
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.01)
import time
# Mini-batch training
batch_size = 4096
print_every = 100
elapsed_time = []
for epoch in range(10000):
    start_time = time.time()
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i+batch_size]
        y = Y[i:i+batch_size]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
    end_time = time.time()
    elapsed_time.append(end_time - start_time)
    if epoch % print_every == 0:
        print(epoch, loss.item())

: 

In [ ]:
# Visualize the embedding

# plot_emb(model.emb, itos)

: 

In [ ]:
from sklearn.manifold import TSNE

def visualize_embeddings_with_tsne(emb, stoi, itos, title='t-SNE Visualization of Embeddings', figsize=(3, 3)):
    """
    Visualizes embeddings with t-SNE.
    
    Args:
    - emb (torch.nn.Embedding): The embedding layer.
    - stoi (dict): Dictionary mapping characters to indices.
    - itos (dict): Dictionary mapping indices to characters.
    - title (str): Title of the plot (default: 't-SNE Visualization of Embeddings').
    - figsize (tuple): Figure size (default: (10, 8)).
    
    Returns:
    - None
    """
    # Get the embeddings from the embedding layer
    embeddings = emb.weight.data.numpy()

    # Initialize t-SNE with desired parameters
    tsne = TSNE(n_components=2, random_state=42)

    # Fit t-SNE to the embeddings
    embeddings_tsne = tsne.fit_transform(embeddings)

    # Visualize the t-SNE embeddings
    plt.figure(figsize=figsize)
    for i in range(len(embeddings)):
        plt.scatter(embeddings_tsne[i, 0], embeddings_tsne[i, 1], label=itos[i], marker='.')
    plt.title(title)
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    # plt.legend(title='Characters')
    plt.show()

# Example usage:
# Assuming you have already defined emb, stoi, and itos
# Replace with your actual embeddings and dictionaries
visualize_embeddings_with_tsne(model.emb, stoi, itos)

: 

In [ ]:
#generate text
print(generate_text(model, itos, stoi, block_size,100))

: 